### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
%load_ext lab_black

In [32]:
import pandas as pd

schools_csv = "Resources/schools_complete.csv"
students_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(schools_csv)
student_df = pd.read_csv(students_csv)

# Combine the data into a single dataset
school_data = pd.merge(
    student_df, school_df, how="left", on=["school_name", "school_name"]
)

school_data = school_data.rename(
    columns={
        "Student ID": "student_id",
        "student_name": "student_name",
        "gender": "gender",
        "grade": "grade",
        "school_name": "school_name",
        "reading_score": "reading_score",
        "math_score": "math_score",
        "School ID": "school_id",
        "type": "school_type",
        "size": "size",
        "budget": "budget",
    }
)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [33]:
tot_schools = school_data["school_name"].nunique()
tot_students = school_data["student_id"].nunique()
tot_budget = school_data["budget"].unique().sum()  # refactor this line
avg_math = school_data["math_score"].mean()
avg_read = school_data["reading_score"].mean()

In [34]:
passed_both = school_data[
    (school_data["math_score"] >= 70) & (school_data["reading_score"] >= 70)
]["student_id"]

overall_pass_rate = (passed_both.nunique() / tot_students) * 100

In [35]:
math_pct_pass = round(
    (
        (
            (
                school_data[school_data["math_score"] >= 70]["math_score"]
                .value_counts()
                .sum()
            )
            / tot_students
        )
        * 100
    ),
    2,
)

reading_pct_pass = round(
    (
        (
            (
                school_data[school_data["reading_score"] >= 70]["reading_score"]
                .value_counts()
                .sum()
            )
            / tot_students
        )
        * 100
    ),
    2,
)

In [36]:
distric_summary_table = pd.DataFrame(
    {
        "tot_schools": tot_schools,  # refactor without []. use index
        "tot_students": tot_students,
        "tot_budget": tot_budget,
        "avg_math": avg_math,
        "avg_read": avg_read,
        "overall_pass_rate": overall_pass_rate,
        "math_pct_pass": math_pct_pass,
        "reading_pct_pass": reading_pct_pass,
    },
    index=[0],
)

In [37]:
distric_summary_table.head()

,tot_schools,tot_students,tot_budget,avg_math,avg_read,overall_pass_rate,math_pct_pass,reading_pct_pass
0,15,39170,24649428,78.985371,81.87784,65.172326,74.98,85.81


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [38]:
school_type = school_data.groupby(["school_name"])["school_type"].first()
school_tot_students = school_data.groupby(["school_name"])["student_id"].nunique()
tot_school_budget = school_data.groupby(["school_name"])["budget"].mean()
per_student_budget = tot_school_budget / school_tot_students

avg_school_math_score = round(
    (school_data.groupby(["school_name"])["math_score"].mean()), 2
)

avg_school_read_score = round(
    (school_data.groupby(["school_name"])["reading_score"].mean()), 2
)

school_pct_pass_math = (
    (
        school_data[school_data["math_score"] >= 70]
        .groupby(["school_name"])["math_score"]
        .count()
    )
    / school_tot_students
) * 100

school_pct_pass_reading = (
    (
        school_data[school_data["reading_score"] >= 70]
        .groupby(["school_name"])["reading_score"]
        .count()
    )
    / school_tot_students
) * 100

# school_overall_pass_rate = (school_pct_pass_math + school_pct_pass_reading) / 2)
# ^ These were the instructions provided, but I changed the calculation to make more practical sense.

school_passed_both = school_data[
    (school_data["math_score"] >= 70) & (school_data["reading_score"] >= 70)
].groupby(["school_name"])["student_id"]

school_overall_pass_rate = (school_passed_both.count() / school_tot_students) * 100

In [39]:
school_summary_table = pd.DataFrame(
    {
        "school_type": school_type,
        "school_tot_students": school_tot_students,
        "tot_school_budget": tot_school_budget,
        "per_student_budget": per_student_budget,
        "avg_school_math_score": avg_school_math_score,
        "avg_school_read_score": avg_school_read_score,
        "school_pct_pass_math": school_pct_pass_math,
        "school_pct_pass_reading": school_pct_pass_reading,
        "school_overall_pass_rate": school_overall_pass_rate,
    },
)
school_summary_table["school_pct_pass_reading"] = school_summary_table[
    "school_pct_pass_reading"
].map("{:.2f}%".format)
school_summary_table["school_pct_pass_math"] = school_summary_table[
    "school_pct_pass_math"
].map("{:.2f}%".format)
school_summary_table["school_overall_pass_rate"] = school_summary_table[
    "school_overall_pass_rate"
].map("{:.2f}%".format)
school_summary_table["tot_school_budget"] = school_summary_table[
    "tot_school_budget"
].map("$ {:}".format)

school_summary_table.head(3)

,school_type,school_tot_students,tot_school_budget,per_student_budget,avg_school_math_score,avg_school_read_score,school_pct_pass_math,school_pct_pass_reading,school_overall_pass_rate
school_name,,,,,,,,,
Bailey High School,District,4976,$ 3124928,628.0,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,$ 1081356,582.0,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,$ 1884411,639.0,76.71,81.16,65.99%,80.74%,53.20%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [40]:
school_summary_table.sort_values(
    by=["school_overall_pass_rate"], ascending=False
).head()

,school_type,school_tot_students,tot_school_budget,per_student_budget,avg_school_math_score,avg_school_read_score,school_pct_pass_math,school_pct_pass_reading,school_overall_pass_rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,$ 1081356,582.0,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,$ 1043130,638.0,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,$ 917500,625.0,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,$ 1319574,578.0,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,$ 585858,609.0,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [41]:
school_summary_table.sort_values(by=["school_overall_pass_rate"]).head()

,school_type,school_tot_students,tot_school_budget,per_student_budget,avg_school_math_score,avg_school_read_score,school_pct_pass_math,school_pct_pass_reading,school_overall_pass_rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,$ 2547363,637.0,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,$ 1884411,639.0,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,$ 1910635,655.0,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,$ 3022020,652.0,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,$ 3094650,650.0,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [42]:
math_mean_9th = (
    school_data.loc[school_data["grade"] == "9th"]
    .groupby(["school_name"])["math_score"]
    .mean()
)
math_mean_10th = (
    school_data.loc[school_data["grade"] == "10th"]
    .groupby(["school_name"])["math_score"]
    .mean()
)
math_mean_11th = (
    school_data.loc[school_data["grade"] == "11th"]
    .groupby(["school_name"])["math_score"]
    .mean()
)
math_mean_12th = (
    school_data.loc[school_data["grade"] == "12th"]
    .groupby(["school_name"])["math_score"]
    .mean()
)

In [53]:
avg_math_by_grade = pd.DataFrame(
    {
        "9th": math_mean_9th,
        "10th": math_mean_10th,
        "11th": math_mean_11th,
        "12th": math_mean_12th,
    }
)
avg_math_by_grade["9th"] = avg_math_by_grade["9th"].map("{:.2f}%".format)
avg_math_by_grade["10th"] = avg_math_by_grade["10th"].map("{:.2f}%".format)
avg_math_by_grade["11th"] = avg_math_by_grade["11th"].map("{:.2f}%".format)
avg_math_by_grade["12th"] = avg_math_by_grade["12th"].map("{:.2f}%".format)

avg_math_by_grade.head(1)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [55]:
reading_mean_9th = (
    school_data.loc[school_data["grade"] == "9th"]
    .groupby(["school_name"])["reading_score"]
    .mean()
)
reading_mean_10th = (
    school_data.loc[school_data["grade"] == "10th"]
    .groupby(["school_name"])["reading_score"]
    .mean()
)
reading_mean_11th = (
    school_data.loc[school_data["grade"] == "11th"]
    .groupby(["school_name"])["reading_score"]
    .mean()
)
reading_mean_12th = (
    school_data.loc[school_data["grade"] == "12th"]
    .groupby(["school_name"])["reading_score"]
    .mean()
)

avg_reading_by_grade = pd.DataFrame(
    {
        "9th": reading_mean_9th,
        "10th": reading_mean_10th,
        "11th": reading_mean_11th,
        "12th": reading_mean_12th,
    }
)
avg_reading_by_grade["9th"] = avg_reading_by_grade["9th"].map("{:.2f}%".format)
avg_reading_by_grade["10th"] = avg_reading_by_grade["10th"].map("{:.2f}%".format)
avg_reading_by_grade["11th"] = avg_reading_by_grade["11th"].map("{:.2f}%".format)
avg_reading_by_grade["12th"] = avg_reading_by_grade["12th"].map("{:.2f}%".format)

avg_reading_by_grade.head(1)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [56]:
spend_cat = pd.cut(
    school_summary_table["per_student_budget"],
    bins=[0, 585, 615, 645, 675],
    labels=["<$585", "$585-615", "$615-645", "$645-675"],
)

In [65]:
spend_per_student_results = pd.DataFrame(
    {
        "Spending_Ranges_(Per Student)": spend_cat,
        "Avg_Math_Score": avg_school_math_score,
        "Avg_Read_Score": avg_school_read_score,
        "%_Passing_Math": school_pct_pass_math,
        "%_Passing_Reading": school_pct_pass_reading,
        "%_Overall_Passing_Rate": school_overall_pass_rate,
    }
)

spend_per_student_results.groupby("Spending_Ranges_(Per Student)").mean()

,Avg_Math_Score,Avg_Read_Score,%_Passing_Math,%_Passing_Reading,%_Overall_Passing_Rate
Spending_Ranges_(Per Student),,,,,
<$585,83.452500,83.935000,93.460096,96.610877,90.369459
$585-615,83.600000,83.885000,94.230858,95.900287,90.216324
$615-645,79.078333,81.891667,75.668212,86.106569,66.112060
$645-675,76.996667,81.026667,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [47]:
school_size = pd.cut(
    school_summary_table["school_tot_students"],
    bins=[0, 1000, 2000, 5000],
    labels=["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"],
)

In [66]:
scores_by_school_size = pd.DataFrame(
    {
        "School_Size": school_size,
        "Avg_Math_Score": avg_school_math_score,
        "Avg_Read_Score": avg_school_read_score,
        "%_Passing_Math": school_pct_pass_math,
        "%_Passing_Reading": school_pct_pass_reading,
        "%_Overall_Passing_Rate": school_overall_pass_rate,
    }
)

scores_by_school_size.groupby("School_Size").mean()

,Avg_Math_Score,Avg_Read_Score,%_Passing_Math,%_Passing_Reading,%_Overall_Passing_Rate
School_Size,,,,,
Small (<1000),83.820,83.92500,93.550225,96.099437,89.883853
Medium (1000-2000),83.374,83.86800,93.599695,96.790680,90.621535
Large (2000-5000),77.745,81.34375,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type.

In [67]:
mapping = {"Charter": 1, "District": 2}
stats_by_type = school_summary_table.replace({"school_type": mapping})

sch_type = pd.cut(stats_by_type["school_type"], bins=2, labels=["Charter", "District"])

In [68]:
scores_by_school_type = pd.DataFrame(
    {
        "School_Type": sch_type,
        "Avg_Math_Score": avg_school_math_score,
        "Avg_Read_Score": avg_school_read_score,
        "%_Passing_Math": school_pct_pass_math,
        "%_Passing_Reading": school_pct_pass_reading,
        "%_Overall_Passing_Rate": school_overall_pass_rate,
    }
)

round((scores_by_school_type.groupby(["School_Type"]).mean()), 1)

,Avg_Math_Score,Avg_Read_Score,%_Passing_Math,%_Passing_Reading,%_Overall_Passing_Rate
School_Type,,,,,
Charter,83.5,83.9,93.6,96.6,90.4
District,77.0,81.0,66.5,80.8,53.7


In [ ]:
Observation 1: Students at Charter schools outperform students at District schools in both reading and math, but by a wider margin in math.
Observation 2: Spending per student doesn't necessarily equate to higher test scores. Scores peak in the $585-$615 range.
Observation 3: There is a sharp distinction in test scores and % passing between schools with less than 2000 students and schools with more than 2000 students.
    Schools with less than 2000 students vastly outperform schools with more than 2000 students.